# Tutorial 01b — Build a Basic Model with PyTorch-Style API

This notebook demonstrates the PyTorch-like API (`nn.Sequential`) for building SOEN models. We'll create a simple 4-layer feedforward network.

---

## 🔊 NOISE CONFIGURATION: ENABLED (Default)

> **This tutorial runs with NOISE INJECTION (documented defaults).**
>
> | Parameter | Default | Description |
> |-----------|---------|-------------|
> | `phi` | **0.01** | Noise on input flux |
> | `s` | **0.005** | Noise on state |
> | `relative` | **False** | Absolute scaling |
>
> **To toggle noise on/off:** Use the `NOISE_ENABLED` variable in the next code cell.

---

## 1) Imports

Import the PyTorch-style API from `soen_toolkit.nn`


In [ ]:
# Setup: Ensure soen_toolkit is importable
import sys
from pathlib import Path

# Add src directory to path if running from notebook location
notebook_dir = Path.cwd()
for parent in [notebook_dir] + list(notebook_dir.parents):
    candidate = parent / "src"
    if (candidate / "soen_toolkit").exists():
        sys.path.insert(0, str(candidate))
        break

import torch

from soen_toolkit import nn
from soen_toolkit.nn.layers import Linear, MultiplierWICC, NonLinear

In [ ]:
# ==============================================================================
# NOISE CONFIGURATION TOGGLE
# ==============================================================================
# Set NOISE_ENABLED = False to run with ideal conditions (no noise)
# Set NOISE_ENABLED = True for noise injection (default)

NOISE_ENABLED = True  # Toggle this to enable/disable noise

# Default noise parameters (documented defaults)
NOISE_DEFAULTS = {
    "phi": 0.01,           # Noise on input flux
    "s": 0.005,            # Noise on state
    "g": 0.0,              # Source function noise
    "bias_current": 0.0,   # Bias current noise
    "j": 0.0,              # Connection weight noise
    "relative": False,     # Absolute scaling
}

def set_model_noise(model, enabled=True, noise_values=None):
    """
    Toggle noise injection on/off for a SOEN model.
    
    Args:
        model: SOENModelCore instance
        enabled: If True, apply noise; if False, set all noise to 0
        noise_values: Dict of noise parameters (uses NOISE_DEFAULTS if None)
    
    Returns:
        model: The modified model (for chaining)
    """
    from soen_toolkit.core.configs import NoiseConfig
    
    if noise_values is None:
        noise_values = NOISE_DEFAULTS
    
    # Update layer noise configurations
    for cfg in model.layers_config:
        if enabled:
            cfg.noise = NoiseConfig(
                phi=noise_values.get("phi", 0.01),
                s=noise_values.get("s", 0.005),
                g=noise_values.get("g", 0.0),
                bias_current=noise_values.get("bias_current", 0.0),
                j=noise_values.get("j", 0.0),
                relative=noise_values.get("relative", False),
                extras=getattr(cfg.noise, "extras", {}),
            )
        else:
            cfg.noise = NoiseConfig(
                phi=0.0, s=0.0, g=0.0, bias_current=0.0, j=0.0,
                relative=False,
                extras=getattr(cfg.noise, "extras", {}),
            )
    
    # Update connection noise configurations
    for conn_cfg in model.connections_config:
        if enabled:
            conn_cfg.noise = NoiseConfig(
                phi=0.0, g=0.0, s=0.0, bias_current=0.0,
                j=noise_values.get("j", 0.0),
                relative=noise_values.get("relative", False),
                extras={},
            )
        else:
            conn_cfg.noise = NoiseConfig(
                phi=0.0, g=0.0, s=0.0, bias_current=0.0, j=0.0,
                relative=False, extras={},
            )
    
    status = "ENABLED" if enabled else "DISABLED"
    print(f"✓ Noise injection {status}")
    if enabled:
        print(f"  phi={noise_values['phi']}, s={noise_values['s']}, "
              f"relative={noise_values['relative']}")
    
    return model

print(f"Noise injection: {'ENABLED' if NOISE_ENABLED else 'DISABLED'}")
if NOISE_ENABLED:
    print(f"  Default values: phi={NOISE_DEFAULTS['phi']}, s={NOISE_DEFAULTS['s']}, "
          f"relative={NOISE_DEFAULTS['relative']}")

## 2) Build the Model

Create a 4-layer network using `nn.Sequential`:
- Layer 0: Linear input layer (3 dimensions)
- Layer 1: MultiplierWICC layer (5 dimensions) - hidden layer using WICC (With Collection Coil) physics
- Layer 2: MultiplierWICC layer (5 dimensions) - hidden layer using WICC physics
- Layer 3: NonLinear output layer (10 dimensions)

Notice how we only specify dimensions and let the system use defaults for everything else (solver, source functions, timestep, etc.).


In [ ]:
model = nn.Sequential(
    [
        Linear(dim=3),  # will be used an input layer
        MultiplierWICC(dim=5),  # WICC (With Collection Coil) multiplier - connections added by default
        MultiplierWICC(dim=5),
        NonLinear(dim=10),  # output layer
    ]
)

## 3) Visualize Architecture (Optional)

Requires Graphviz to be installed. This generates a visual diagram of the network.


In [ ]:
model.visualize(show_desc=True)

## 4) Run a Forward Pass

Create a random input and simulate the network dynamics.


In [ ]:
# Create input: (batch_size, time_steps, input_dim)
x = torch.randn(1, 50, 3)

# Run forward pass
output = model(x)

## 5) Model Summary

Print a summary showing layers and parameter counts.


In [ ]:
model.summary(notebook_view=True)

---

That's it! You've built and simulated a basic SOEN model using the PyTorch.nn-style API.

**Next steps:**
- Try modifying the layer dimensions
- Add more layers
- Check out Tutorial 02 for training models
